In [115]:
#The path to the biopython packages was not automatically added to anaconda. To allow import of the package, a manual path must be specified.
import os
#Absolute location: 

#os.environ['CONDA_PREFIX']=/home/wish1832/anaconda3
#Code to append biopython location to path
os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+"/pkgs/biopython-1.78-py38h7b6447c_0/lib/python3.8/site-packages"

#### Based on analysis in R, Uniq2 (alignment[1]) contains the reference sequence. 

# Scaffolding Code for Variant Calling

## Functions

In [155]:
from Bio import AlignIO as msa_in
from Bio.Align import MultipleSeqAlignment as msa
import Bio.SeqRecord
import numpy as np
import pandas as pd

def initalize_files(outname):
    """
    output = an output file prefix (string)
    
    Given an output file prefix, initialize_files() will create three files (outname_substitutions.tsv, outname_insertions.tsv, outname_deletions.tsv) 
    containing raw data on the substitutions, insertions, and deletions relative to the reference genome found in the sequence alignment. 
    The function will print headers in each file in the order specified below:
    
    Field 1: Type of variant (insertion, deletion, substitution)
    Field 2: Position of variant in the multiple sequence alignment
    Field 3: Position of variant in reference sequence (for insertions, this is the two positions the insertion ocurrs between.)
    Field 4: Amino Acid of Reference Strand (for insertions, this is the two amino acid residues the insertion ocurrs between.)
    Field 5: Variant Amino Acid Residue
    Field 6: Cluster_ID (e.g. clust48)
    Field 7: Number of sequences in cluster
    """
    sub_out = "../Variant_Call/"+outname+"_substitutions_raw.tsv"
    ins_out = "../Variant_Call/"+outname+"_insertions_raw.tsv"
    del_out = "../Variant_Call/"+outname+"_deletions_raw.tsv"
    header = ["Type","Nuc_Pos(MSA)","Nuc_Pos(Ref)","Ref Residue","Var Residue","Cluster_ID","N_sequences_in_cluster"]

    #Print header in output file
    with open(raw_out,"w") as outfile:
        outfile.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(header[0],header[1],header[2],header[3],header[4],header[5],header[6]))

    """Code below will prevent overwriting in final version.
        if not file:
            with open(file,"w") as file
        else:
            print("Error: {} already exists. Overwriting prohibited.".format(file))"""
    return sub_out,ins_out,del_out

def reference_position(reference):
    """
    For Variant Code Creation, the positions must correspond to the reference sequence, rather than the alignent.
    
    Given the reference genome, reference_position() will return a list of integers that stores the reference genome position corresponding to the msa position.
    If there is no base in the reference genome at a given position in the msa (when a "-" is in the reference genome), a blank is stored in this list.
    """
    ref_list=[]
    pos=1
    for i in reference:
        if i != "-":
            ref_list.append(pos)
            pos += 1
        else:
            ref_list.append("-")
    return ref_list        
        
def insertion_flanks(reference,ref_list,msadex):
    """
    reference: the reference genome, as a subset of the msa (biopython Seq object)
    ref_list: a list used to fetch the position on the reference genome corresponding to the msa position (list)
    msadex: the base in the msa queried in the current iteration (i, integer)
    
    For insertion events, this function will yield both the position and the identity of the two residues on the reference 
    strand that the insertion ocurrs between.
    
    Returns two elements as a tuple: 
    refpos, refbase
    
    refpos: the positions of the flanking amino acids to the new insertion (on the reference genome)
    refbase: the one-letter codes of the flanking amino acids on the reference genome
    
    tuple assignment should be used to store the two variables. e.g.
    refpos,refbase=insertion_flanks(i)
    """
    #First, the function will count down from the multiple sequence alignment index in the reference genome position 
    while ref_list[msadex]=="-":
        msadex-=1
        #SPECIAL CASE
        #If the insertion ocurrs at the beginning of the reference genome, the special case below is triggered to prevent indexing below zero.
        #The output will be the position (1) and the identity of the first AA of the reference strain, with no range. 
        if msadex<=0:
            #Iterate msa index up by one and test to see if there is a position on the reference position list
            #that corresponds to the msa index. If not, continue iteration
            msadex+=1
            while ref_list[msadex]=="-":
                msadex+=1
            refpos=ref_list[msadex]
            ref_base=reference[msadex]
            return refpos,refbase
        #END SPECIAL CASE
    #When the first flanking AA has been found, the identity and position of the amino acid on the reference genome is recorded
    base_lower=reference[msadex]
    lower=ref_list[msadex]
    #Next, the function counts up one to the beginning of the "-" in the reference genome and keeps counting up the msa index until an extant amino acid residue is found
    msadex+=1
    while ref_list[msadex]=="-":
        #Iterate msa index up by one and test to see if there is a position on the reference position list
        #that corresponds to the msa index. If not, continue iteration
        msadex+=1
        #Prevents msadex from being greater than the max msa index (avoiding an index error)
        if msadex>=len(alignment[0].seq):
            #If an insertion ocurrs after the end of the reference sequence, there is no upper bound.
            #The output will be the amino acid and its position at the end of the reference genome, with no range.
            refpos=lower
            refbase=base_lower
            return refpos,refbase
    #When the upper flank AA has been found, the identity and position of the amino acid on the reference genome is recorded
    base_upper=reference[msadex]
    upper=ref_list[msadex]
    refpos="{}-{}".format(lower,upper)
    refbase="{}-{}".format(base_lower,base_upper)
    print("Insertion is between reference bases {} and {}. The flanking residues are {} and {}.".format(lower,upper,base_lower,base_upper))
    return refpos, refbase
    
def store_base_info(vartype,ref_list,reference,base,alignment,i,j,outfile,insertion=False):
    """
    When a variant is detected, this function will store the info associated with the variant.
    """ 
    #Store position of variant (in MSA)
    position=i+1
    #Store position of variant (in Reference strand). For insertions, this will be stored as a range (the two bases the insertion is between)
    if insertion==False:
        refpos=ref_list[i]
    #if the event is an insertion, both the reference positions and the reference bases will be determined by the same function
    elif insertion==True:
        refpos,refbase=insertion_flanks(reference,ref_list,i)
    #Store Base on Reference Strand (if not an insertion)
    if insertion==False:
        refbase=reference[i]
    #Store Base on Variant Strand
    varbase=base[j]
    #Retrieve name of cluster with variant (before ";" in id)
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster (printed betweeen ";" and "=" in id)
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]
    #Write data to tsv file
    write_line([vartype,position,refpos,refbase,varbase,clustname,clustsize],outfile)

def msa_iterator(alignment,raw_out,reference,ref_list):
    for i in range(len(alignment[0].seq)):
        base=alignment[:,i]
        #i is the amino acid position, j is the sequence position in the MSA
        #Base is a string that contains the amino acid at position i for each of the j clusters in the alignment
        for j in range(len(base)):
            #Conditional to detect a substitution
            if reference[i] != base[j] and reference[i]!="-" and base[j]!="-":
                print("AA substitution: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],base[j],j+1))
                store_base_info("sub",ref_list,reference,base,alignment,i,j,raw_out)

            #Conditional to detect a deletion
            elif reference[i] != base[j] and base[j]=="-":
                print("AA deletion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],base[j],j+1))
                store_base_info("del",ref_list,reference,base,alignment,i,j,raw_out)

            #Conditional to detect an insertion
            elif reference[i] != base[j] and reference[i]=="-":
                print("AA insertion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],base[j],j+1))
                store_base_info("ins",ref_list,reference,base,alignment,i,j,raw_out,insertion=True)
                
"""
Part 2: Functions for analysis of raw variant data and report production
"""
def initialize_report_files(outname):
    """
    outname = the directory and file prefix of the output files. Specific files will contain 
    
    intitialize_report_files will create four files containing reports on the mutations present.
    subs_report,ins_report,and dels_report will contain files
    """
    subs_report=outname+"_substitutions.txt"
    ins_report=outname+"_insertions.txt"
    dels_report=outname+"_deletions.txt"
    clusters_report=outname+"_clusters.txt"
    
    #Create blank files to be filled with information in later steps 
    for file in [subs_report,ins_report,dels_report,clusters_report]:
        with (open(file,"w")) as outfile:
                outfile.write("")
    return subs_report,ins_report,dels_report,clusters_report



In [159]:
def initalize_file(outname):
    """
    outname = the directory and prefix of the output file (string)
        For example, setting outname "mydir/mutant" will create an outfile in the directory mydir with the name mutant_variants_raw.tsv.
        Other reports generated by the output will also exist in the directory mydir and have the prefix "mutant".
    
    Given an output file prefix, initialize_files() will create one file (outname_variants_raw.tsv) 
    containing raw data on the substitutions, insertions, and deletions relative to the reference genome found in the sequence alignment. 
    
    The function will print headers in each file in the order specified below:
    
    Field 1: Cluster_ID (e.g. clust48)  
    Field 2: Number of sequences in cluster 
    Field 3: Type of variant (insertion, deletion, substitution)
    Field 4: Mutation code (formatted based on the nomenclature indicated by the Human Genome Variation Society https://varnomen.hgvs.org/recommendations/protein/)
    Field 5: Amino Acid(s) of Reference Strand 
        For insertions, this is the two amino acid residues the insertion ocurrs between.
        For deletions, this will be the reference residue(s) deleted 
    Field 6: Variant Amino Acid Residue
        For multi-codon insertions, this field will contain a string of multiple codons
        For deletions, this will be blank
    Field 7: Start position of variant on multiple sequence alignment
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
    Field 8: End position of variant on multiple sequence alignment 
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    Field 9: Start position of variant on reference strand 
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
    Field 10: End position of variant on reference strand
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    """
    #Create filename for raw output
    raw_out = outname+"_variants_raw.tsv"
    #Define header according to elements listed in docstring
    header = ["Cluster_ID","Cluster_Size","Type","Code","Ref Residue(s)","Var Residue(s)","AA_Start(MSA)","AA_End(MSA)","AA_Start(Ref)","AA_End(Ref)"]

    #Print header in output file
    with open(raw_out,"w") as outfile:
        outfile.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(header[0],header[1],header[2],header[3],header[4],header[5],header[6],header[7],header[8],header[9]))

    """Code below will prevent overwriting in final version.
        if not file:
            with open(file,"w") as file
        else
        
            print("Error: {} already exists. Overwriting prohibited.".format(file))"""
    #Return the filename for use in other functions
    return raw_out

def write_line(outfile,clustname,clustsize,var_type,code,ref_AA,var_AA,msa_start,ref_start,msa_end="-",ref_end="-"):
    """
    Will write out to the variant .tsv values given the location of an output list with 6 Fields in the following format:
    
    Field 1: Cluster_ID (e.g. clust48)  
    Field 2: Number of sequences in cluster 
    Field 3: Type of variant (insertion, deletion, substitution)
    Field 4: Mutation code (formatted based on the nomenclature indicated by the Human Genome Variation Society https://varnomen.hgvs.org/recommendations/protein/)
    Field 5: Amino Acid(s) of Reference Strand 
        For insertions, this is the two amino acid residues the insertion ocurrs between.
        For deletions, this will be the reference residue(s) deleted 
    Field 6: Variant Amino Acid Residue
        For multi-codon insertions, this field will contain multiple columns
        For deletions, this will be blank
    Field 7: Start position of variant on multiple sequence alignment
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
    Field 8: End position of variant on multiple sequence alignment 
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    Field 9: Start position of variant on reference strand 
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
    Field 10: End position of variant on reference strand
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    """
    #Write to output file
    if outfile:
        with open(outfile,"a") as file:
            file.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(clustname,clustsize,var_type,code,ref_AA,var_AA,msa_start,msa_end,ref_start,ref_end))
    
    #Raise a FileNotFoundError if the output file has not been initialized
    else:
        raise FileNotFoundError("{} does not exist.".format(outfile))

def msa_iterator1(j_begin,i_begin,alignment,raw_out,reference,ref_index):
    """
    alignment=a biopython multiple sequence alignment object
    raw_out=filename for outputting raw data
    reference=Reference strand to which sequences are compared (biopython RefSeq object)
    ref_list=A list relating an amino acid position on a multiple sequence alingment to its corresponding position on the reference strand
    i_begin=Amino acid position on multiple sequence alignment at which to begin iteration (allows for skipping over multi-base insertions and deletions)
    j_begin=Alignment object at which to begin
    
    
    msa_iterator() will iterate through the multiple sequence alignment object and compare bases to the reference genome. 
    Insertions, deletions, and substitutions will be detected and handled by separate functions based on type. 
    
    i is the amino acid position, j is the sequence position in the MSA (origional iterated through AA position first)
    """
    #Every sequence in the alignment will be iteratively compared to the reference strand
    for j in range(j_begin,len(alignment)): 
        #No longer necessary: base=alignment[:,i]
        #Base is a string that contains the amino acid at position i for each of the j clusters in the alignment
        
        for i in range(i_begin,len(alignment[0].seq)):
            print("testing j={}, i={}".format(j,i))
            #Conditional to detect a substitution
            if reference[i] != alignment[j,i] and reference[i]!="-" and alignment[j,i]!="-":
                print("AA substitution: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
                substitution_case(i,j,alignment,reference,ref_index,raw_out)
                
            #Conditional to detect a deletion
            elif (reference[i] != alignment[j,i]) and alignment[j,i]=="-":
                print("AA deletion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
                print("j={}, i={}".format(j,i))
                j,i=deletion_case(j,i,alignment,reference,ref_index)
                #After the code for deletion_case is ran, "return" stops both for loops.
                #The function will be restarted at the i,j values at the end of the deletion case.
                return j,i
                #This statement tells the computer to stop iteration through the current i,j range 
                #and focus on the iteration in the substitution case. For multi-base events, this is necessary to
                #prevent the function from recording the same event multiple times.

            #Conditional to detect an insertion
            elif reference[i] != alignment[j,i] and reference[i]=="-":
                print("AA insertion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
    
    #If the end of both loops is reached, return the j,i values to break the while loop in the main function
    return j,i

def substitution_case(i,j,alignment,reference,ref_index,outfile):
    """
    Will activate when a substution is detected during iteration through the multiple sequence alignment. 
    """
    #Store position of variant in MSA (codon position is one more than the index)
    msa_pos=i+1
    #Store position of variant in reference genome
    ref_pos=ref_index[i]
    #Store codon in position i of reference genome
    ref_AA=reference[i]
    #Store the variant codon
    var_AA=alignment[j,i] #Retrieves the codon from the sequence at position j, at position i
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]
    #Store the substitution code
    #Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
    #The code format is <Reference_Amino_Acid><Codon_Position(Reference Genome)><Variant_Amino_Acid>
    #Source (https://varnomen.hgvs.org/recommendations/protein/variant/substitution/)
    code="{}{}{}".format(ref_AA,ref_pos,var_AA)
    #Write above information to file
    #write_line([clustname,clustsize,"sub",code,msa_pos,ref_pos,ref_AA,var_AA],outfile)
    #TEMP:print function used below in place of write_line
    print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(clustname,clustsize,"sub",code,msa_pos,ref_pos,ref_AA,var_AA))
    #No iteration skipping is required since all substitution events are recorded one base at a time, even when adjacent substitutions are present
    #Once information is stored, return to iteration at the next base (returns to the For loop in the parent function)

def deletion_case(j,i,alignment,reference,ref_index):
    """
    Will activate when a deletion is detected during iteration through the multiple sequence alignment. 
    
    Since deletions may span two or more bases and nomenclature counts multi-base deletions as a single event, 
    iteration through the multiple sequence alignment in the parent msa_iterator() function must be skipped ahead
    to the end of the detion region to avoid a multiple-codon deletion being read as multiple instances of a single-base deletion.
    After recording a deletion, msa_iterator() is restarted at the position i corresponding to the codon after the end of the deletion. 
    """
    #Raise an error if the function is triggered by a codon that is not deleted
    if not (alignment[j,i]=="-" and reference[i]!="-"):
        raise ValueError("Deletion case called on a codon region without a deletion")
    #Or if the deletion case is not called at the beginning of a deletion region
    #Code detects a deletion in the codon one index before the current i it is called on
    elif i!=0 and ((reference[i-1]!=alignment[j,i-1]) and alignment[j,i-1]=="-"): 
        raise ValueError("Deletion case called in the middle of a multi-base deletion")

    #Record the position at the beginning of a deletion
    i_start=i   
   
    #Iterate forward in strain j until a codon is found, or until a special case is triggered
    for i in range(i,len(alignment[0].seq)):
        #If a deletion case is detected, try the next base
        #Another case to skip would be where both the genome j and the reference do not have a base (due to an insertion on another strand)
        #i must not correspond to the end of the sequence to trigger this conditional
        if ((alignment[j,i]=="-" and reference[i]!="-") or (alignment[j,i]=="-" and reference[i]=="-")) and(i!=len(reference)-1):
            print("Skipping blank at position {}".format(i+1))
        #Special case: it is possible for an insertion to ocurr with a deletion (delins event).
            #TODO: function for delins event
        elif alignment[j,i]!="-" and reference[i]=="-":
            print("Found a delins event.")
            return j,i
        #Special case: tail end of sequence is deleted (no codon found at the end of the MSA)
        elif i==(len(reference)-1):
            print("Reached end of genome. Deletion ocurrs from index {} until the end of the genome.".format(i_start))
            #Iterate backward until the last index in the reference genome is found 
            while ref_index[i]=="-":
                i-=1
                print("In while loop. i decreased by one.")
            #when the last index is found, store the reference genome position corresponding to that position.
            print("last reference base found. i_end={}".format(i))
            i_end=i
            record_deletion_info(j,i_start,i_end,alignment,reference,ref_index)
            #In this special case, iteration must be restarted at i=0 on the next genome to avoid triggering an error in msa_iterator()
            if j!=len(alignment)-1:
                return j+1,0
            else:
                return j,len(reference)-1
            
        #when a codon is found, store the position corresponding to the end of the deletion region
        #Because of possible cases where both the ref genome and the genome j both have no codons at the end of a deletion region,
        #The correct index for the end of the deletion region should correspond to the last index within the deletion region where 
        #the ref. genome has a base.         
        elif alignment[j,i]!="-" and reference[i]!="-":
            print("discovered codon at position {}.".format(i+1))
            print("index of extant codon: {}".format(i))
            #TODO: Change i_end (add while loop)
            i_end=i-1
            print("Setting i_end={}".format(i_end))
            #Pass the start and end positions to record_deletion info to print info to file
            record_deletion_info(j,i_start,i_end,alignment,reference,ref_index)
            #Return current j,i positions to pick up iteration in the msa_iterator() function
            return j,i
        
        else:
            print("a case was found that does not satisfy the defined conditionals.")
            return j,i
        
def record_deletion_info(j,i_start,i_end,alignment,reference,ref_index):
    """
    Information recording
    """
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]

    #Conditional: for single codon deletions, positions and amino acid residues will be stored as a single entry
    #For multi-codon deletions, data will be stored as a range
    #For single-codon deletions
    if i_start==i_end:
        #Store position of variant in MSA
        msa_start=i_end+1
        msa_end="-"
        #Store position of variant in reference genome
        ref_start=ref_index[i_end]
        ref_end="-"
        #Store codon in position i of reference genome
        ref_AA=reference[i_end]
        #Store the variant codon
        var_AA="-" #Retrieves the codon from the sequence at position j, at position i    
        #Store the deletion code
        """
        Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
        the code format is <Amino_Acid_Deleted><Position_of_Deleted_Codon, reference Genome>del
        source: (https://varnomen.hgvs.org/recommendations/protein/variant/deletion/)
        """
        code="{}{}del".format(ref_AA,ref_start)
    #For multi-codon deletions
    else:
        #Store position range in MSA
        msa_start=i_start+1
        msa_end=i_end+1

        #Store position range in reference genome
        ref_start=ref_index[i_start]
        ref_end=ref_index[i_end]

        #Store reference amino acids that have been deleted in sequence j
        #Slicing does not include the end value, so the end value should be one index after the end of the deletion region (i)
        ref_AA=deletion_ref_AA(i_start,i_end,reference)

        print(ref_AA)
        #Store a blank for deleted variant codons
        var_AA="-"
        print(var_AA)
        #Store the deletion code
        """
        Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
        the code format is <Beginning_Amino_Acid><Beginning_Codon_Position, reference Genome>_<Ending_Amino_Acid><Ending_Codon_Position>del
        source: (https://varnomen.hgvs.org/recommendations/protein/variant/deletion/)
        """
        code="{}{}_{}{}del".format(reference[i_start],ref_index[i_start],reference[i_end],ref_index[i_end])
    #Write above information to file
    #write_line([clustname,clustsize,"sub",code,msa_pos,ref_pos,ref_AA,var_AA],outfile)
    #TEMP:print function used below in place of write_line
    print("Cluster_ID","Cluster_Size","Type","Code","Ref Residue(s)","Var Residue(s)","AA_Start(MSA)","AA_End(MSA)","AA_Start(Ref)","AA_End(Ref)",sep="\t")
    print(clustname,clustsize,"del",code,ref_AA,var_AA,msa_start,msa_end,ref_start,ref_end,sep="\t")
        
def deletion_ref_AA(i_start,i_end,reference):
    """
    For multi-base deletions, records the refernce AA's deleted without blanks that may appear in the aligned reference genome.
    
    This function should only be called on multi-base deletions, otherwise an error will occur
    """
    ref_AA=reference[i_start]
    #the amino acid at i_start is included first, and iteration goes up to and including i_end (must be i_end+1 because range() does not include stop value)
    for i in range(i_start+1,i_end+1):
        #If an amino acid exists on the reference genome (no blank on msa), add the AA to the sequence
        if reference[i]!="-":
            ref_AA=ref_AA+reference[i]
    return ref_AA

In [4]:
def output_clusters_by_code(df,outfile):
    #Iterator: loops through all unique codes 
    for i in range(len(df.code.unique())):
        sub_df = df[df.code==df.code.unique()[i]]
        #Create empty lists to store all cluster IDs with the variant in question, and the size of each cluster
        clusters=[]
        clust_sizes=[]
        #Loop through all entries in the dataframe that match the current cluster ID
        for item in sub_df["Cluster_ID"]:
            #Add the cluster ID to the cluster list
            clusters.append(item)
            #Find the size of the cluster ID and add that to the list for cluster sizes
            clust_sizes.append(sub_df.loc[sub_df["Cluster_ID"]==item,"N_sequences_in_cluster"].iat[0])
        #Once the lists for the cluster IDs and sizes for the variant type have been created, iterate through each list and add it to the report file.
        #First, the total number of sequences with the given variant is computed and added to the file next to the variant code (variant header).
        total_seq=np.array(clust_sizes).sum()
        #Write variant header information to file
        with (open(outfile,"a")) as file:
            file.write(">{}|{} sequences".format(df.code.unique()[i],total_seq))
            #Iterate through the cluster names and sizes in the lists and add information to file
            for j in range(len(clusters)):
                file.write(clusters[j]," size "+str(clust_sizes[j]),sep=":")
            #Add a line break at the end of the information for the current code
            file.write("\n")

In [106]:
reference[3:5]+reference[5]

Seq('FLV')

In [134]:
reference[1319]

'-'

In [105]:
reference[3:6]

Seq('FLV')

In [117]:
test_genome=reference[3]
for i in range(3+1,6,1):
    print(test_genome)
    test_genome=test_genome+reference[i]
    print(test_genome)

F
FL
FL
FLV


In [118]:
test_genome

'FLV'

## Program Execution

In [160]:
"""
Fake test AA sequence to check function of program
"""
#Load MSA, store as biopython multiple sequence alingment object
alignment = msa_in.read("msa_tester.fasta","fasta")

#Initialize files
#Define output file name
outname = "../Variant_Call/test_tester"
raw_out = initalize_file(outname)

#Define reference genome (here it is index 1 of the multiple sequence alignment)
reference=alignment[0].seq
#Create reference sequence object storing reference sequence indeces corresponding to msa index (if "-" in reference seq after alignment)
ref_index=reference_position(reference)

#Iterate through multiple sequence alignment and write raw data from variant information to file
j,i=msa_iterator1(0,0,alignment,raw_out,reference,ref_index)

#If a multi-codon insertion or deletion breaks the loop, restart the iterator unless on the last base.
while j<(len(alignment)-1) and i<(len(reference)-1):
    if i==len(reference)-1:
        print("MSA carriage return")
        j,i=msa_iterator(j+1,0,alignment,raw_out,reference,ref_index)
    print("Restarting with j={} and i={}".format(j,i))
    j,i=msa_iterator1(j,i,alignment,raw_out,reference,ref_index)

testing j=0, i=0
testing j=0, i=1
testing j=0, i=2
testing j=0, i=3
testing j=0, i=4
testing j=0, i=5
testing j=0, i=6
testing j=0, i=7
testing j=0, i=8
testing j=0, i=9
testing j=0, i=10
testing j=0, i=11
testing j=0, i=12
testing j=0, i=13
testing j=0, i=14
testing j=0, i=15
testing j=1, i=0
AA insertion: position 1 in cluster 2: -1A
testing j=1, i=1
AA insertion: position 2 in cluster 2: -2E
testing j=1, i=2
AA insertion: position 3 in cluster 2: -3D
testing j=1, i=3
AA insertion: position 4 in cluster 2: -4K
testing j=1, i=4
AA insertion: position 5 in cluster 2: -5E
testing j=1, i=5
testing j=1, i=6
testing j=1, i=7
testing j=1, i=8
testing j=1, i=9
testing j=1, i=10
testing j=1, i=11
testing j=1, i=12
testing j=1, i=13
testing j=1, i=14
testing j=1, i=15
testing j=2, i=0
testing j=2, i=1
AA insertion: position 2 in cluster 3: -2E
testing j=2, i=2
AA insertion: position 3 in cluster 3: -3D
testing j=2, i=3
testing j=2, i=4
testing j=2, i=5
testing j=2, i=6
testing j=2, i=7
testing

ValueError: Deletion case called in the middle of a multi-base deletion

In [154]:
"""
Part 1: Generate raw variant data from multiple sequence alignment input
"""
#Load MSA, store as biopython multiple sequence alingment object
alignment = msa_in.read("../Aligned_Sequences/spikeprot0902_full_aligned_nosingletons_1iter.fasta",'fasta')

#Initialize files
#Define output file name
outname = "../Variant_Call/test"
raw_out = initalize_file(outname)

#Define reference genome (here it is index 1 of the multiple sequence alignment)
reference=alignment[1].seq
#Create reference sequence object storing reference sequence indeces corresponding to msa index (if "-" in reference seq after alignment)
ref_index=reference_position(reference)

#Iterate through multiple sequence alignment and write raw data from variant information to file
j,i=msa_iterator1(0,0,alignment,raw_out,reference,ref_index)

#If a multi-codon insertion or deletion breaks the loop, restart the iterator unless on the last base.
while j<(len(alignment)-1) and i<(len(reference)-1):
    if i==len(reference)-1:
        print("MSA carriage return")
        j,i=msa_iterator(j+1,0,alignment,raw_out,reference,ref_index)
    print("Restarting with j={} and i={}".format(j,i))
    j,i=msa_iterator1(j,i,alignment,raw_out,reference,ref_index)

AA substitution: position 660 in cluster 1: D660G
Uniq1	38860	sub	D614G	660	614	D	G
AA substitution: position 523 in cluster 3: S523N
Uniq3	3058	sub	S477N	523	477	S	N
AA substitution: position 660 in cluster 3: D660G
Uniq3	3058	sub	D614G	660	614	D	G
AA substitution: position 660 in cluster 4: D660G
Uniq4	2520	sub	D614G	660	614	D	G
AA substitution: position 1186 in cluster 4: P1186X
Uniq4	2520	sub	P1140X	1186	1140	P	X
AA substitution: position 255 in cluster 5: S255X
Uniq5	879	sub	S255X	255	255	S	X
AA substitution: position 256 in cluster 5: S256X
Uniq5	879	sub	S256X	256	256	S	X
AA substitution: position 257 in cluster 5: G257X
Uniq5	879	sub	G257X	257	257	G	X
AA substitution: position 258 in cluster 5: W258X
Uniq5	879	sub	W258X	258	258	W	X
AA substitution: position 259 in cluster 5: T259X
Uniq5	879	sub	T259X	259	259	T	X
AA substitution: position 260 in cluster 5: A260X
Uniq5	879	sub	A260X	260	260	A	X
AA substitution: position 261 in cluster 5: G261X
Uniq5	879	sub	G261X	261	261	G	X
AA su

Uniq8	555	sub	L303X	313	303	L	X
AA substitution: position 314 in cluster 8: K314X
Uniq8	555	sub	K304X	314	304	K	X
AA substitution: position 315 in cluster 8: S315X
Uniq8	555	sub	S305X	315	305	S	X
AA substitution: position 316 in cluster 8: F316X
Uniq8	555	sub	F306X	316	306	F	X
AA substitution: position 317 in cluster 8: T317X
Uniq8	555	sub	T307X	317	307	T	X
AA substitution: position 318 in cluster 8: V318X
Uniq8	555	sub	V308X	318	308	V	X
AA substitution: position 319 in cluster 8: E319X
Uniq8	555	sub	E309X	319	309	E	X
AA substitution: position 320 in cluster 8: K320X
Uniq8	555	sub	K310X	320	310	K	X
AA substitution: position 321 in cluster 8: G321X
Uniq8	555	sub	G311X	321	311	G	X
AA substitution: position 322 in cluster 8: I322X
Uniq8	555	sub	I312X	322	312	I	X
AA substitution: position 323 in cluster 8: Y323X
Uniq8	555	sub	Y313X	323	313	Y	X
AA substitution: position 324 in cluster 8: Q324X
Uniq8	555	sub	Q314X	324	314	Q	X
AA substitution: position 325 in cluster 8: T325X
Uniq8	555	sub	T3

AA substitution: position 660 in cluster 16: D660G
Uniq16	307	sub	D614G	660	614	D	G
AA substitution: position 1309 in cluster 16: P1309L
Uniq16	307	sub	P1263L	1309	1263	P	L
AA substitution: position 660 in cluster 17: D660G
Uniq17	276	sub	D614G	660	614	D	G
AA substitution: position 301 in cluster 18: C301X
Uniq18	276	sub	C291X	301	291	C	X
AA substitution: position 302 in cluster 18: A302X
Uniq18	276	sub	A292X	302	292	A	X
AA substitution: position 303 in cluster 18: L303X
Uniq18	276	sub	L293X	303	293	L	X
AA substitution: position 304 in cluster 18: D304X
Uniq18	276	sub	D294X	304	294	D	X
AA substitution: position 305 in cluster 18: P305X
Uniq18	276	sub	P295X	305	295	P	X
AA substitution: position 306 in cluster 18: L306X
Uniq18	276	sub	L296X	306	296	L	X
AA substitution: position 307 in cluster 18: S307X
Uniq18	276	sub	S297X	307	297	S	X
AA substitution: position 308 in cluster 18: E308X
Uniq18	276	sub	E298X	308	298	E	X
AA substitution: position 309 in cluster 18: T309X
Uniq18	276	sub	T299X

Uniq24	190	sub	C525X	571	525	C	X
AA substitution: position 572 in cluster 24: G572X
Uniq24	190	sub	G526X	572	526	G	X
AA substitution: position 573 in cluster 24: P573X
Uniq24	190	sub	P527X	573	527	P	X
AA substitution: position 574 in cluster 24: K574X
Uniq24	190	sub	K528X	574	528	K	X
AA substitution: position 365 in cluster 25: R365X
Uniq25	171	sub	R355X	365	355	R	X
AA substitution: position 366 in cluster 25: K366X
Uniq25	171	sub	K356X	366	356	K	X
AA substitution: position 367 in cluster 25: R367X
Uniq25	171	sub	R357X	367	357	R	X
AA substitution: position 368 in cluster 25: I368X
Uniq25	171	sub	I358X	368	358	I	X
AA substitution: position 369 in cluster 25: S369X
Uniq25	171	sub	S359X	369	359	S	X
AA substitution: position 370 in cluster 25: N370X
Uniq25	171	sub	N360X	370	360	N	X
AA substitution: position 371 in cluster 25: C371X
Uniq25	171	sub	C361X	371	361	C	X
AA substitution: position 372 in cluster 25: V372X
Uniq25	171	sub	V362X	372	362	V	X
AA substitution: position 373 in cluster 25

AA substitution: position 307 in cluster 35: S307X
Uniq35	101	sub	S297X	307	297	S	X
AA substitution: position 308 in cluster 35: E308X
Uniq35	101	sub	E298X	308	298	E	X
AA substitution: position 309 in cluster 35: T309X
Uniq35	101	sub	T299X	309	299	T	X
AA substitution: position 310 in cluster 35: K310X
Uniq35	101	sub	K300X	310	300	K	X
AA substitution: position 311 in cluster 35: C311X
Uniq35	101	sub	C301X	311	301	C	X
AA substitution: position 312 in cluster 35: T312X
Uniq35	101	sub	T302X	312	302	T	X
AA substitution: position 313 in cluster 35: L313X
Uniq35	101	sub	L303X	313	303	L	X
AA substitution: position 314 in cluster 35: K314X
Uniq35	101	sub	K304X	314	304	K	X
AA substitution: position 315 in cluster 35: S315X
Uniq35	101	sub	S305X	315	305	S	X
AA substitution: position 316 in cluster 35: F316X
Uniq35	101	sub	F306X	316	306	F	X
AA substitution: position 317 in cluster 35: T317X
Uniq35	101	sub	T307X	317	307	T	X
AA substitution: position 318 in cluster 35: V318X
Uniq35	101	sub	V308X	318	

...

AA substitution: position 660 in cluster 458: D660G
Uniq458	8	sub	D614G	660	614	D	G
AA substitution: position 660 in cluster 459: D660G
Uniq459	8	sub	D614G	660	614	D	G
AA substitution: position 989 in cluster 459: S989I
Uniq459	8	sub	S943I	989	943	S	I
AA substitution: position 660 in cluster 462: D660G
Uniq462	8	sub	D614G	660	614	D	G
AA substitution: position 825 in cluster 462: Q825H
Uniq462	8	sub	Q779H	825	779	Q	H
AA substitution: position 660 in cluster 463: D660G
Uniq463	7	sub	D614G	660	614	D	G
AA substitution: position 704 in cluster 463: N704Y
Uniq463	7	sub	N658Y	704	658	N	Y
AA substitution: position 660 in cluster 464: D660G
Uniq464	7	sub	D614G	660	614	D	G
AA substitution: position 716 in cluster 464: I716M
Uniq464	7	sub	I670M	716	670	I	M
AA substitution: position 485 in cluster 466: N485X
Uniq466	7	sub	N439X	485	439	N	X
AA substitution: position 486 in cluster 466: N486X
Uniq466	7	sub	N440X	486	440	N	X
AA substitution: position 487 in cluster 466: L487X
Uniq466	7	sub	L441X	487	

AA substitution: position 1186 in cluster 475: P1186X
Uniq475	7	sub	P1140X	1186	1140	P	X
AA substitution: position 660 in cluster 476: D660G
Uniq476	7	sub	D614G	660	614	D	G
AA substitution: position 485 in cluster 477: N485X
Uniq477	7	sub	N439X	485	439	N	X
AA substitution: position 486 in cluster 477: N486X
Uniq477	7	sub	N440X	486	440	N	X
AA substitution: position 487 in cluster 477: L487X
Uniq477	7	sub	L441X	487	441	L	X
AA substitution: position 488 in cluster 477: D488X
Uniq477	7	sub	D442X	488	442	D	X
AA substitution: position 489 in cluster 477: S489X
Uniq477	7	sub	S443X	489	443	S	X
AA substitution: position 490 in cluster 477: K490X
Uniq477	7	sub	K444X	490	444	K	X
AA substitution: position 491 in cluster 477: V491X
Uniq477	7	sub	V445X	491	445	V	X
AA substitution: position 492 in cluster 477: G492X
Uniq477	7	sub	G446X	492	446	G	X
AA substitution: position 493 in cluster 477: G493X
Uniq477	7	sub	G447X	493	447	G	X
AA substitution: position 494 in cluster 477: N494X
Uniq477	7	sub	N448X

Skipping blank at position 1052
Skipping blank at position 1053
Skipping blank at position 1054
Skipping blank at position 1055
Skipping blank at position 1056
Skipping blank at position 1057
Skipping blank at position 1058
Skipping blank at position 1059
Skipping blank at position 1060
Skipping blank at position 1061
Skipping blank at position 1062
Skipping blank at position 1063
Skipping blank at position 1064
Skipping blank at position 1065
Skipping blank at position 1066
Skipping blank at position 1067
Skipping blank at position 1068
Skipping blank at position 1069
Skipping blank at position 1070
Skipping blank at position 1071
Skipping blank at position 1072
Skipping blank at position 1073
Skipping blank at position 1074
Skipping blank at position 1075
Skipping blank at position 1076
Skipping blank at position 1077
Skipping blank at position 1078
Skipping blank at position 1079
Skipping blank at position 1080
Skipping blank at position 1081
Skipping blank at position 1082
Skipping

AA substitution: position 660 in cluster 483: D660G
Uniq483	7	sub	D614G	660	614	D	G
AA substitution: position 925 in cluster 483: A925S
Uniq483	7	sub	A879S	925	879	A	S
AA substitution: position 1186 in cluster 483: P1186X
Uniq483	7	sub	P1140X	1186	1140	P	X
AA substitution: position 499 in cluster 484: Y499F
Uniq484	7	sub	Y453F	499	453	Y	F
AA substitution: position 660 in cluster 485: D660G
Uniq485	7	sub	D614G	660	614	D	G
AA deletion: position 301 in cluster 486: C301-
j=485, i=300


ValueError: Deletion case called in the middle of a multi-base deletion

In [153]:
msa_iterator1(485,0,alignment,raw_out,reference,ref_index)

AA deletion: position 1 in cluster 486: M1-
j=485, i=0
Skipping blank at position 1
Skipping blank at position 2
Skipping blank at position 3
Skipping blank at position 4
Skipping blank at position 5
Skipping blank at position 6
Skipping blank at position 7
Skipping blank at position 8
Skipping blank at position 9
Skipping blank at position 10
Skipping blank at position 11
Skipping blank at position 12
Skipping blank at position 13
Skipping blank at position 14
Skipping blank at position 15
Skipping blank at position 16
Skipping blank at position 17
Skipping blank at position 18
Skipping blank at position 19
Skipping blank at position 20
Skipping blank at position 21
Skipping blank at position 22
Skipping blank at position 23
Skipping blank at position 24
Skipping blank at position 25
Skipping blank at position 26
Skipping blank at position 27
Skipping blank at position 28
Skipping blank at position 29
Skipping blank at position 30
Skipping blank at position 31
Skipping blank at positi

(485, 439)

In [152]:
alignment[485,1:310]

SeqRecord(seq=Seq('------------------------------------------------------...---'), id='Uniq486;size=7;', name='Uniq486;size=7;', description='Uniq486;size=7;', dbxrefs=[])

In [147]:
reference[280:310]

Seq('-NGTITDAVDCALDPLSETK')

In [135]:
alignment[284,1319]

'-'

In [127]:
#Code breaker
#Cluster has deletions at the end of the sequence
alignment[282,750:1382]

SeqRecord(seq=Seq('V-----------------------------------------------------...---'), id='Uniq283;size=13;', name='Uniq283;size=13;', description='Uniq283;size=13;', dbxrefs=[])

In [126]:
len(alignment[0])

1320

In [85]:
#Discovered an indel event
reference[394:404]

Seq('----STFKCY')

In [84]:
#Discovered an indel event
alignment[34,394:404]

SeqRecord(seq=Seq('-------KCY'), id='Uniq35;size=101;', name='Uniq35;size=101;', description='Uniq35;size=101;', dbxrefs=[])

In [4]:
#Step 2: Report information from raw file
variants = pd.read_csv("../Variant_Call/test_variants_raw.tsv",delimiter="\t")

#TODO: outname should be put into a function (likely the main function)
outname="../Variant_Call/test"
#Initialize output file
subs_report,ins_report,dels_report,clusters_report=initialize_report_files(outname)

"""
Reporting Substitutions
"""
#Subset for substitutions
subs=variants[variants.Type=="sub"]
#Row numbers are based on the origional table. Must reset them to match the substitution table for facilitated coding
subs=subs.reset_index()
#Create a new column for the substitution code and fill values (format <Reference Amino Acid><Reference Position><Variant Amino Acid>)
for i in range(len(subs)):
    subs.loc[i,"code"]="{}{}{}".format(subs.loc[i,"Ref Residue"],subs.loc[i,"Nuc_Pos(Ref)"],subs.loc[i,"Var Residue"])


In [21]:
#Code for a multi-level filter
sample=variants[(variants["Type"]=="del")&(variants["Cluster_ID"]=="Uniq1745")]

In [41]:
len(alignment[0])

1320

In [48]:
alignment[734]

SeqRecord(seq=Seq('-FFFFLHFALVNSQCVNLTGRAAIQPSFTNSSQRGVYYPDTIFRSNTLVLSQGY...YT-'), id='Uniq735;size=4;', name='Uniq735;size=4;', description='Uniq735;size=4;', dbxrefs=[])

In [22]:
sample

,Type,Nuc_Pos(MSA),Nuc_Pos(Ref),Ref Residue,Var Residue,Cluster_ID,N_sequences_in_cluster
56,del,1,1,M,-,Uniq1745,2
139,del,2,2,F,-,Uniq1745,2
221,del,3,3,V,-,Uniq1745,2
306,del,4,4,F,-,Uniq1745,2
418,del,5,5,L,-,Uniq1745,2
511,del,6,6,V,-,Uniq1745,2
594,del,7,7,L,-,Uniq1745,2
680,del,8,8,L,-,Uniq1745,2
766,del,9,9,P,-,Uniq1745,2
846,del,10,10,L,-,Uniq1745,2


In [26]:
if len(sample[(sample["Nuc_Pos(Ref)"]==17)])==0:
    print("Does not exist")

Does not exist


In [29]:
#Code to query the existence of a residue position (will be iterative)
if not 17 in sample["Nuc_Pos(Ref)"]:
    print("Value does not exist")

Value does not exist


In [12]:
dels.Cluster_ID.unique()[10:15]

array(['Uniq1745', 'Uniq1748', 'Uniq1985', 'Uniq1994', 'Uniq2102'],
      dtype=object)

In [10]:
len(dels.Cluster_ID.unique())

564

In [339]:
"""
Write out substitutions
"""
#Print number of unique codes and skip a line before printing information for each code
with open(subs_report,"w") as file: 
    print("Number of unique substitutions in input: ",len(subs.code.unique()),sep="",end="\n\n",file=file)


2193

In [329]:


#Iterator: loops through all unique codes 
for i in range(len(subs.code.unique())):
    sub_subs = subs[subs.code==subs.code.unique()[i]]
    #Create empty lists to store all cluster IDs with the variant in question, and the size of each cluster
    clusters=[]
    clust_sizes=[]
    #Loop through all entries in the dataframe that match the current cluster ID
    for item in sub_subs["Cluster_ID"]:
        #Add the cluster ID to the cluster list
        clusters.append(item)
        #Find the size of the cluster ID and add that to the list for cluster sizes
        clust_sizes.append(sub_subs.loc[sub_subs["Cluster_ID"]==item,"N_sequences_in_cluster"].iat[0])
    #Once the lists for the cluster IDs and sizes for the variant type have been created, iterate through each list and add it to the report file.
    #First, the total number of sequences with the given variant is computed and added to the file next to the variant code (variant header).
    total_seq=np.array(clust_sizes).sum()
    #Write variant header information to file
    print(">{}|{} sequences".format(subs.code.unique()[i],total_seq))
    #Iterate through the cluster names and sizes in the lists and add information to file
    for j in range(len(clusters)):
        print(clusters[j]," size "+str(clust_sizes[j]),sep=":")
    #Add a line break at the end of the information for the current code
    print("")

Number of unique substitutions in input: 2193

>M1X|456 sequences
Uniq63: size 50
Uniq90: size 36
Uniq92: size 36
Uniq112: size 31
Uniq134: size 26
Uniq140: size 25
Uniq173: size 21
Uniq214: size 17
Uniq262: size 14
Uniq315: size 12
Uniq328: size 11
Uniq329: size 11
Uniq364: size 10
Uniq460: size 8
Uniq475: size 7
Uniq515: size 7
Uniq592: size 6
Uniq615: size 5
Uniq621: size 5
Uniq639: size 5
Uniq641: size 5
Uniq667: size 5
Uniq679: size 5
Uniq710: size 5
Uniq718: size 5
Uniq773: size 4
Uniq895: size 4
Uniq1005: size 3
Uniq1062: size 3
Uniq1173: size 3
Uniq1209: size 3
Uniq1242: size 3
Uniq1247: size 3
Uniq1332: size 3
Uniq1347: size 3
Uniq1378: size 2
Uniq1402: size 2
Uniq1413: size 2
Uniq1438: size 2
Uniq1452: size 2
Uniq1507: size 2
Uniq1561: size 2
Uniq1566: size 2
Uniq1627: size 2
Uniq1703: size 2
Uniq1721: size 2
Uniq1750: size 2
Uniq1811: size 2
Uniq1818: size 2
Uniq1830: size 2
Uniq1839: size 2
Uniq1887: size 2
Uniq1914: size 2
Uniq1926: size 2
Uniq1938: size 2
Uniq1971: size 2

>P9X|470 sequences
Uniq63: size 50
Uniq90: size 36
Uniq92: size 36
Uniq112: size 31
Uniq134: size 26
Uniq140: size 25
Uniq173: size 21
Uniq214: size 17
Uniq262: size 14
Uniq315: size 12
Uniq328: size 11
Uniq329: size 11
Uniq350: size 10
Uniq364: size 10
Uniq460: size 8
Uniq475: size 7
Uniq515: size 7
Uniq592: size 6
Uniq615: size 5
Uniq621: size 5
Uniq639: size 5
Uniq641: size 5
Uniq667: size 5
Uniq679: size 5
Uniq710: size 5
Uniq718: size 5
Uniq773: size 4
Uniq787: size 4
Uniq895: size 4
Uniq1005: size 3
Uniq1026: size 3
Uniq1062: size 3
Uniq1173: size 3
Uniq1209: size 3
Uniq1226: size 3
Uniq1242: size 3
Uniq1247: size 3
Uniq1332: size 3
Uniq1347: size 3
Uniq1378: size 2
Uniq1402: size 2
Uniq1413: size 2
Uniq1438: size 2
Uniq1452: size 2
Uniq1507: size 2
Uniq1566: size 2
Uniq1627: size 2
Uniq1703: size 2
Uniq1721: size 2
Uniq1750: size 2
Uniq1811: size 2
Uniq1818: size 2
Uniq1839: size 2
Uniq1887: size 2
Uniq1914: size 2
Uniq1926: size 2
Uniq1938: size 2
Uniq1971: size 2
Uniq1980: siz

Uniq1402: size 2
Uniq1413: size 2
Uniq1423: size 2
Uniq1438: size 2
Uniq1452: size 2
Uniq1507: size 2
Uniq1627: size 2
Uniq1666: size 2
Uniq1703: size 2
Uniq1721: size 2
Uniq1745: size 2
Uniq1750: size 2
Uniq1811: size 2
Uniq1818: size 2
Uniq1839: size 2
Uniq1887: size 2
Uniq1914: size 2
Uniq1926: size 2
Uniq1938: size 2
Uniq1971: size 2
Uniq1980: size 2
Uniq1994: size 2
Uniq2010: size 2
Uniq2032: size 2
Uniq2107: size 2
Uniq2124: size 2
Uniq2140: size 2
Uniq2168: size 2
Uniq2208: size 2
Uniq2264: size 2
Uniq2311: size 2

>N17Y|2 sequences
Uniq2028: size 2

>L18X|508 sequences
Uniq63: size 50
Uniq90: size 36
Uniq92: size 36
Uniq112: size 31
Uniq134: size 26
Uniq140: size 25
Uniq173: size 21
Uniq214: size 17
Uniq229: size 15
Uniq262: size 14
Uniq315: size 12
Uniq328: size 11
Uniq329: size 11
Uniq350: size 10
Uniq364: size 10
Uniq460: size 8
Uniq475: size 7
Uniq515: size 7
Uniq592: size 6
Uniq615: size 5
Uniq621: size 5
Uniq627: size 5
Uniq639: size 5
Uniq641: size 5
Uniq667: size 5
Uniq

>P25X|519 sequences
Uniq63: size 50
Uniq90: size 36
Uniq92: size 36
Uniq112: size 31
Uniq134: size 26
Uniq140: size 25
Uniq173: size 21
Uniq214: size 17
Uniq229: size 15
Uniq262: size 14
Uniq315: size 12
Uniq328: size 11
Uniq329: size 11
Uniq350: size 10
Uniq364: size 10
Uniq460: size 8
Uniq475: size 7
Uniq515: size 7
Uniq592: size 6
Uniq615: size 5
Uniq621: size 5
Uniq627: size 5
Uniq639: size 5
Uniq641: size 5
Uniq667: size 5
Uniq679: size 5
Uniq710: size 5
Uniq718: size 5
Uniq789: size 4
Uniq882: size 4
Uniq895: size 4
Uniq961: size 3
Uniq1005: size 3
Uniq1029: size 3
Uniq1173: size 3
Uniq1209: size 3
Uniq1226: size 3
Uniq1242: size 3
Uniq1247: size 3
Uniq1261: size 3
Uniq1332: size 3
Uniq1347: size 3
Uniq1378: size 2
Uniq1402: size 2
Uniq1413: size 2
Uniq1423: size 2
Uniq1438: size 2
Uniq1452: size 2
Uniq1507: size 2
Uniq1620: size 2
Uniq1621: size 2
Uniq1627: size 2
Uniq1666: size 2
Uniq1703: size 2
Uniq1709: size 2
Uniq1721: size 2
Uniq1745: size 2
Uniq1750: size 2
Uniq1811: size

Uniq2107: size 2
Uniq2124: size 2
Uniq2140: size 2
Uniq2168: size 2
Uniq2198: size 2
Uniq2208: size 2
Uniq2264: size 2
Uniq2311: size 2

>F32X|523 sequences
Uniq63: size 50
Uniq90: size 36
Uniq92: size 36
Uniq112: size 31
Uniq134: size 26
Uniq140: size 25
Uniq173: size 21
Uniq214: size 17
Uniq229: size 15
Uniq262: size 14
Uniq315: size 12
Uniq328: size 11
Uniq329: size 11
Uniq350: size 10
Uniq355: size 10
Uniq364: size 10
Uniq460: size 8
Uniq475: size 7
Uniq515: size 7
Uniq592: size 6
Uniq615: size 5
Uniq621: size 5
Uniq627: size 5
Uniq633: size 5
Uniq639: size 5
Uniq641: size 5
Uniq651: size 5
Uniq667: size 5
Uniq679: size 5
Uniq710: size 5
Uniq718: size 5
Uniq882: size 4
Uniq895: size 4
Uniq961: size 3
Uniq1005: size 3
Uniq1173: size 3
Uniq1182: size 3
Uniq1209: size 3
Uniq1226: size 3
Uniq1242: size 3
Uniq1247: size 3
Uniq1261: size 3
Uniq1332: size 3
Uniq1347: size 3
Uniq1378: size 2
Uniq1402: size 2
Uniq1413: size 2
Uniq1418: size 2
Uniq1423: size 2
Uniq1438: size 2
Uniq1452: size

...

>S1097X|13 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1620: size 2
Uniq1626: size 2

>N1098X|15 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1620: size 2
Uniq1626: size 2
Uniq2290: size 2

>G1099X|13 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1797: size 2

>G1099D|2 sequences
Uniq2276: size 2

>T1100X|11 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>H1101Y|25 sequences
Uniq185: size 20
Uniq1268: size 3
Uniq2250: size 2

>H1101X|11 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>W1102X|11 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>W1102L|2 sequences
Uniq2310: size 2

>F1103X|11 sequences
Uniq891: size 4
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>V1104L|33 sequences
Uniq226: size 16
Uniq1147: size 3
Uniq1173: size 3
Uniq1226: size 3
Uniq1701: size 2
Uniq1839: size 2
Uniq2032: size 2
Uniq2261: size 2

>V

>D1153X|9 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1620: size 2
Uniq1626: size 2

>D1153A|3 sequences
Uniq1232: size 3

>D1153Y|2 sequences
Uniq1959: size 2

>K1154X|9 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1620: size 2
Uniq1626: size 2

>Y1155X|9 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1620: size 2
Uniq1626: size 2

>F1156X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>F1156L|3 sequences
Uniq1263: size 3

>K1157X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>K1157N|2 sequences
Uniq1632: size 2

>N1158X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>H1159Y|6 sequences
Uniq897: size 4
Uniq1461: size 2

>H1159X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>T1160I|5 sequences
Uniq996: size 3
Uniq1517: size 2

>T1160X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>S1161X|7 sequences
Uniq1000: size 3
Uniq1557: size 2
Uniq1626: size 2

>P1162L|45 sequences
Uniq101: size 34
Uni

Uniq1721: size 2
Uniq1938: size 2

>N1187Y|27 sequences
Uniq137: size 25
Uniq1524: size 2

>N1187X|64 sequences
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2

>E1188X|64 sequences
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2

>V1189X|64 sequences
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2

>V1189I|2 sequences
Uniq1520: size 2

>A1190X|64 sequences
Uniq315: size 12
Uniq549: size 6
Uniq

>F1220X|95 sequences
Uniq123: size 29
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2
Uniq2018: size 2

>I1221X|93 sequences
Uniq123: size 29
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2

>I1221T|2 sequences
Uniq1598: size 2

>A1222X|93 sequences
Uniq123: size 29
Uniq315: size 12
Uniq549: size 6
Uniq562: size 6
Uniq572: size 6
Uniq615: size 5
Uniq683: size 5
Uniq920: size 4
Uniq1000: size 3
Uniq1039: size 3
Uniq1155: size 3
Uniq1328: size 3
Uniq1557: size 2
Uniq1626: size 2
Uniq1721: size 2
Uniq1938: size 2

>G1223X|95 sequences
Uniq123: size 29
Uniq315: size 12
Uniq549: size 6
Uniq562: s

>C1248X|47 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1620: size 2
Uniq1626: size 2
Uniq1938: size 2
Uniq2234: size 2

>C1248S|5 sequences
Uniq719: size 5

>S1249X|45 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1626: size 2
Uniq1938: size 2
Uniq2234: size 2

>S1249Y|4 sequences
Uniq883: size 4

>C1250X|44 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1329: size 3
Uniq1626: size 2

>C1250F|9 sequences
Uniq721: size 5
Uniq1385: size 2
Uniq1975: size 2

>G1251X|41 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1626: size 2

>G1251V|22 sequences
Uniq295: size 12
Uniq423: size 8
Uniq2260: size 2

>G1251R|5 sequences
Uniq647: size 5

>S1252P|56 sequences
Uniq81: size 39
Uniq275: size 13
Uniq870: size 4

>S1252X|41 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1626: size 2

>S1252F|10 sequences
Uniq450: size 8
Uniq2039: size 2

>C1253X|41 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1626: size 2

>C1254X|43 sequences
Uniq123: size 29
Uniq373: size 10
Uniq1626: size 2
Uniq1675

In [320]:
len(subs.code.unique())

2193

In [64]:
test_list=["a","-","-","-","-","-","-","-","b","c","d"]

In [73]:
#Iteration skip without recursion
def non_recursive_iterator(i_begin,test_list,end=False):
    for i in range(i_begin,len(test_list)):
        if test_list[i]!="-":
            print(test_list[i])
        elif test_list[i]=="-":
            print("found a blank at index {}".format(i))
            continue

In [74]:
non_recursive_iterator(0,test_list)

a
found a blank at index 1
found a blank at index 2
found a blank at index 3
found a blank at index 4
found a blank at index 5
found a blank at index 6
found a blank at index 7
b
c
d


In [73]:
test_list[2]!="-"

False

In [6]:
#Model for multi-base event recording
def mini_iterator(i_begin,test_list):
    for i in range(i_begin,len(test_list)):
        if test_list[i]!="-":
            print(test_list[i])
        elif test_list[i]=="-":
            print("found a blank at index {}".format(i))
            i_blank=i
            skipper(i_blank,test_list)
            break #Break the for loop (critical to prevent recording the event more than once)
            
def skipper(i_blank,test_list):
    for i in range(i_blank+1,len(test_list)):
        if test_list[i] == "-":
            continue
        elif test_list[i] != "-":
            print("blanks discovered between indeces {} and {}".format(i_blank,i))
            mini_iterator(i,test_list) 
            break
          

In [81]:
mini_iterator(0,test_list)

a
found a blank at index 1
blanks discovered between indeces 1 and 8
b
c
d


In [131]:
for n in test:
    print(n)
    if n>10:
        next(n)
        next(n)

6
22


TypeError: 'int' object is not an iterator

In [142]:
c=iter(test)
for n in c:
    print(n)
    if n>10:
        for i in range(2):
            print("next iteration")
            next(c)

6
22
next iteration
next iteration
5
2


In [86]:
test=[6,22,22,4,5,2]

In [87]:
for a in test:
    print(a)
    if a>10:
        print("%s is greater than 10" % a)
        test.remove(test[test.index(a)])

6
22
22 is greater than 10
4
5
2


In [88]:
test

[6, 22, 4, 5, 2]

In [92]:
#Carrige retrun example (for progress meter)
for x in range(100000001):
    if x%100000 == 0:
        print('{}\r'.format(x), end="")
print()

100000000
